<a href="https://colab.research.google.com/github/evyshen/DATASCI_112/blob/main/DATASCI_112_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

:

In [ ]:
pip install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 26.5 MB/s eta 0:00:00


In [ ]:
# Install Chrome and Chromium Driver
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
!sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'
!sudo apt-get update
!sudo apt-get install -y google-chrome-stable
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium webdriver-manager

# Selenium setup with proper binary location
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# ChromeOptions setup
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = '/usr/bin/google-chrome'

OK
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,200 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,719 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-b

In [ ]:
# Launch browser
# Sometimes have to run this block twice to get it to work without throwing an error
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                          options = options)

# Click 'Show More' until all professors are visible
driver.get('https://www.ratemyprofessors.com/search/professors/953?q=*')

while True:
    try:
        show_more_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Show More')]")))
        show_more_button.click()
        time.sleep(0.1)

    except:
        break

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Scrape all professor links
base_url = 'https://www.ratemyprofessors.com'
professor_links = []
ratings = []

for link in soup.find_all('a',
                          class_ = 'TeacherCard__StyledTeacherCard-syjs0d-0',
                          href = True):
    full_url = base_url + link['href']
    professor_links.append(full_url)

I could not FOR THE LIFE OF ME figure out how to get more than the first twenty reviews to load! I tried for a couple hours, but was unsuccessful :(
  
I left the last code block that I thought would work below (commented out)

In [ ]:
# TEST
# driver = webdriver.Chrome(options = options)

# professors_data = []

# driver.get('https://www.ratemyprofessors.com/professor/134687')

# while True:
#   try:
#     show_more_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Load More Ratings')]")))
#     show_more_button.click()
#     time.sleep(1.5)
#   except:
#     break

# soup = BeautifulSoup(driver.page_source, 'html.parser')

#   # Extract name
# name_span = soup.find('span', class_='HeaderDescription__NameWrapper-sc-1lt205f-5')
# full_name = name_span.get_text(separator=' ', strip=True)

#   # Extract comments using regex
# comments = re.findall(r'"comment":"(.*?)"', soup.prettify())

# professors_data.append({'Professor': full_name, 'Student Reviews': comments})
#   # time.sleep(0.1)

In [ ]:
import re

headers = {'User-Agent' :
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) \
           AppleWebKit/605.1.15 (KHTML, like Gecko) \
           Version/18.5 Safari/605.1.15'}

professors_data = []

for url in professor_links:
  response = requests.get(url, headers = headers)

  # Parse page source with BeautifulSoup
  soup = BeautifulSoup(response.text, 'html.parser')

  # Extract name
  name_span = soup.find('span', class_='HeaderDescription__NameWrapper-sc-1lt205f-5')
  full_name = name_span.get_text(separator=' ', strip=True)

  # Extract comments using regex
  comments = re.findall(r'"comment":"(.*?)"', soup.prettify())

  # Extract rating using regex
  rating_match = re.search(r'"avgRating":([\d.]+)', soup.prettify())
  rating = float(rating_match.group(1)) if rating_match else None


  # Extract number of ratings
  num_ratings_match = re.search(r'"numRatings":(\d+)', soup.prettify())
  num_ratings = int(num_ratings_match.group(1)) if num_ratings_match else None

  professors_data.append({'Professor': full_name, 'Student Reviews': comments, 'Overall Rating': rating, 'Num Ratings': num_ratings})
  # time.sleep(0.1)

  # Should probably add a time.sleep() but I didn't yet because it takes a super long time to run the code because there are 400+ professors

KeyboardInterrupt: 

In [ ]:
# Create DataFrame
df = pd.DataFrame(professors_data)
df = df.sort_values("Num Ratings", ascending = False)
df

In [ ]:
# Identify broken links
base_url = 'https://profiles.stanford.edu/'

stanford_profiles_links= []

stanford_profiles_df_links = []

for professor in df['Professor']:
  url = base_url + professor.lower().split(' ')[0] + '-' + professor.lower().split(' ')[1]
  stanford_profiles_links.append(url)

for url in stanford_profiles_links:
    response = requests.get(url, headers = headers)
    if response.status_code == 404:
        stanford_profiles_df_links.append(None)
    else:
        stanford_profiles_df_links.append(url)

df["Stanford Profile"] = stanford_profiles_df_links

In [ ]:
df['Number of RMP Reviews'] = df['Student Reviews'].apply(len)
sorted_review_df = df.sort_values('Number of RMP Reviews', ascending = False)
sorted_review_df = sorted_review_df.set_index('Professor')
sorted_review_df

In [ ]:
valid_sorted_review_df = sorted_review_df[~sorted_review_df["Stanford Profile"].isna()]
valid_sorted_review_df

In [ ]:
no_empty_comments_df = valid_sorted_review_df[valid_sorted_review_df["Number of RMP Reviews"] != 0]
no_empty_comments_df["Student Reviews"] = no_empty_comments_df["Student Reviews"].apply(
    lambda reviews: [r for r in reviews if r.strip().lower() != "no comments"]
)
no_empty_comments_df

In [ ]:
departments = []
positions = []

for url in valid_sorted_review_df["Stanford Profile"]:
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")

  dept_tag = soup.find("span", class_="department")
  pos_tag = soup.find("span", class_="position")
  if pos_tag:
      pos_text = pos_tag.get_text(strip=True)
      if "Position:" in pos_text:
          # Extra cleanup in case there's leftover unicode or HTML entities
          pos_text = pos_text.replace("Position:", "").replace("\xa0", "").strip()
      positions.append(pos_text)
  else:
      positions.append("Not found")

  if dept_tag:
      dept_text = dept_tag.get_text(strip=True)
      if "Department:" in dept_text:
          # Extra cleanup in case there's leftover unicode or HTML entities
          dept_text = dept_text.replace("Department:", "").replace("\xa0", "").strip()[5:]
      departments.append(dept_text)
  else:
      departments.append("Not found")

# Add results to your DataFrame
valid_sorted_review_df["Department"] = departments

# Add results to your DataFrame
valid_sorted_review_df["Position"] = positions

valid_sorted_review_df

In [ ]:
#publications

publication_counts = []

for url in valid_sorted_review_df["Stanford Profile"]:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    pub_tags = soup.find_all("li", class_="publication article")
    count = len(pub_tags)
    publication_counts.append(count)

# Add the counts to your DataFrame
valid_sorted_review_df["Num Publications"] = publication_counts

valid_sorted_review_df


In [ ]:
# published

def has_publications(publication_counts):
    if publication_counts > 0:
        return "Yes"
    return "No"

valid_sorted_review_df["Published"] = valid_sorted_review_df["Num Publications"].apply(has_publications)
valid_sorted_review_df

In [ ]:
sp_titles = []

for url in valid_sorted_review_df["Stanford Profile"]:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the <h2> tag
    h2_tag = soup.find("h2")
    if h2_tag:
        sp_titles.append(h2_tag.get_text(strip=True))
    else:
        sp_titles.append("Not found")

# Add the title column to your DataFrame
valid_sorted_review_df["SP Title"] = sp_titles


In [ ]:
#published

def is_tenured(row):
    position = row["Position"]
    sp_title = row["SP Title"]

    # Check if either string contains "professor"
    if (
        isinstance(position, str) and "professor" in position.lower()
    ) or (
        isinstance(sp_title, str) and "professor" in sp_title.lower()
    ):
        return "Yes"
    return "No"

# Apply to rows (use axis=1 to access multiple columns per row)
valid_sorted_review_df["Tenured"] = valid_sorted_review_df.apply(is_tenured, axis=1)


In [ ]:
url = "https://dlsun.github.io/pods/data/names/yob2000.txt"

gender_names_df = pd.read_csv(url,
                              header=None,
                              names=['First Name', 'Sex', 'Count'])

# Sort so the most frequent gender per name comes first
gender_names_df_sorted = gender_names_df.sort_values(['First Name', 'Count'],
                                                     ascending = [True, False])

# Drop duplicates by name, keeping the first (i.e., most frequent) gender
gender_names_df_cleaned = gender_names_df_sorted.drop_duplicates(subset = 'First Name',
                                                                 keep = 'first')
# Don't need the 'Count' column anymore
gender_names_df_cleaned = gender_names_df_sorted.drop('Count', axis = 1)

gender_names_df_cleaned

In [ ]:
# Merge DataFrames
valid_sorted_review_df = valid_sorted_review_df.reset_index()

valid_sorted_review_df['First Name'] = valid_sorted_review_df['Professor'].str.split(' ').str[0]

prof_gender_df = pd.merge(valid_sorted_review_df,
                                  gender_names_df_cleaned,
                                  on = 'First Name',
                                  how = 'left')

prof_gender_df

In [ ]:
# Check which professors did not get assigned a gender
prof_gender_df[prof_gender_df['Sex'].isna()]

In [ ]:
# Drop Uta Francke: her comments are 'no comments'
prof_gender_df = prof_gender_df[prof_gender_df['Professor'] != 'Uta Francke']

In [ ]:
# Check which professors did not get assigned a gender
prof_gender_df[prof_gender_df['Sex'].isna()]

# Manually fill gender for those not yet assigned
manual_gender = pd.DataFrame({
    'Professor': [
        'Mehran Sahami',
        'Ewart Thomas',
        'Wray Huestis',
        'Liran Einav',
        'Chaitan Khosla',
        'Pavle Levi',
        'Wojciech Wieczorek',
        'Jure Leskovec',
        'Blakey Vermeule',
        'Usha Iyer',
        'Agripino Silveira',
        'Dafna Zur',
        'Hongjie Dai',
        'Tadashi Fukami',
        'Persi Diaconis',
        'Guosong Hong',
        'Ge Wang',
        'Mutallip Anwar',
        'Subhasish Mitra'
    ],
    'Sex': [
        'M',
        'M',
        'F',
        'M',
        'M',
        'M',
        'M',
        'M',
        'F',
        'F',
        'M',
        'F',
        'M',
        'M',
        'M',
        'M',
        'M',
        'M',
        'M'
    ]
})

# Fill missing sex values in valid_sorted_review_df with sex values from manual_gender
prof_gender_df['Sex'] = prof_gender_df['Sex'].fillna(
    prof_gender_df['Professor'].map(manual_gender.set_index('Professor')['Sex'])
)


In [ ]:
# Check that there are no missing sex values, should return zero
(prof_gender_df['Sex'].isna()).sum()

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1CWp7OneH_1xv845LUwHkgQJPPRnHL9Z-/export?format=csv&gid=922383119'
num_degrees_conferred = pd.read_csv(url)

num_degrees_conferred = num_degrees_conferred.rename(columns={'Unnamed: 0': 'Department'})

# Fix the fact that some degree names didn't fill in because of the data structure
num_degrees_conferred['Department'] = num_degrees_conferred['Department'].ffill()
num_degrees_conferred['2023-24'] = num_degrees_conferred['2023-24'].fillna(0)

department_size = num_degrees_conferred.groupby('Department')['2023-24'].sum().reset_index()
department_size = department_size.rename(columns={'2023-24': 'Department Size'})

In [ ]:
department_size

In [ ]:
# Merge DataFrames
dep_size_df = pd.merge(prof_gender_df,
                       department_size,
                       on = 'Department',
                       how = 'left')

In [ ]:
# Check for missing department sizes
list(dep_size_df[dep_size_df['Department Size'].isna()]['Department'].unique())

In [ ]:
manual_dep_sizes = pd.DataFrame({
    'Department': [
 'Mechanical Engineering - Flow Physics and Computation',
 'Graduate School of Education',
 'History Department',
 'Pathology Stem Cell Institute (XSCI)',
 'German',
 'Art & Art History',
 'Electrical Engineering - Information Systems Laboratory',
 'Law School',
 'Sustainable Architecture & Engineering',
 'Department of Geophysics - Geophysics',
 'Radiation Oncology - Radiation Physics',
 'Medicine - Med/Oncology',
 'Emergency Medicine - Adult Clinical/Academic',
 'Earth Systems Program',
 'Pediatrics - Infectious Diseases',
 'East Asian Languages and Cultures',
 'Center for Race & Ethnicity',
 'Graduate School of Business - Faculty',
 'Energy Science & Engineering',
 'Department of Developmental Biology'],
    'Department Size' : [
              261,
              171,
              38,
              79,
              6,
              31,
              230,
              287,
              206,
              16,
              79,
              79,
              79,
              16,
              79,
              33,
              18,
              455,
              3,
              7]
})

# Fill missing, if manual data collection available
dep_size_df['Department Size'] = dep_size_df['Department Size'].fillna(
    dep_size_df['Department'].map(manual_dep_sizes.set_index('Department')['Department Size']))

# There are still about 50 that are unknown, but I did the best that I could :(
dep_size_df

In [ ]:
#removed duplicates

dep_size_df = dep_size_df.drop_duplicates(subset="Professor", keep="first")
dep_size_df = dep_size_df.reset_index(drop=True)
dep_size_df=dep_size_df.drop(columns = "index")

In [ ]:
dep_size_df

In [ ]:
merged_df = dep_size_df.merge(df, on="Professor", how="left")
merged_df = merged_df[~(merged_df["Num Ratings"] == 0)]
merged_df = merged_df.drop(columns = "Student Reviews_y")
merged_df = merged_df.rename(columns={'Student Reviews_x': 'Student Reviews'})
merged_df.sort_values("Num Ratings")
merged_df = merged_df.reset_index(drop=True)
merged_df

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

sentiments = []

for i in range(len(merged_df)):
  text = merged_df.loc[i, 'Student Reviews']
  score = sia.polarity_scores(str(text))
  sentiments.append(score['compound'])

# print(sentiment)  # Returns compound, pos, neu, neg scores

'''
compound score is a normalized, weighted sum that you can use to classify sentiment:

  compound >= 0.05 → Positive; +1.0 = most positive
  compound <= -0.05 → Negative; –1.0 = most negative
  Otherwise → Neutral; 0.0 = neutral

'''
merged_df['Rating Sentiment'] = sentiments


In [ ]:
final_df = merged_df

In [ ]:
from google.colab import files
final_df.to_csv("final_df.csv", index=False)
files.download("final_df.csv")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

In [ ]:
# Linear regression for the association between sex and rating sentiment

pipeline = make_pipeline(
    OneHotEncoder(drop = 'first'),  # avoid multicollinearity
    LinearRegression()
)

pipeline.fit(dep_size_df[['Sex']], dep_size_df['Rating Sentiment'])

y_pred = pipeline.predict(dep_size_df[['Sex']])

encoder = pipeline.named_steps['onehotencoder']
model = pipeline.named_steps['linearregression']

feature_names = encoder.get_feature_names_out(['Sex'])

coefficients = pd.Series(model.coef_, index=feature_names)

print("Intercept:", model.intercept_)
print("Coefficients:")
print(coefficients)

In [ ]:
# Linear regression for the association between department size and rating sentiment

clean_df = dep_size_df[~dep_size_df['Department Size'].isna()]

model = LinearRegression()

model.fit(clean_df[['Department Size']], clean_df['Rating Sentiment'])

# Predictions
y_pred = model.predict(clean_df[['Department Size']])

intercept = model.intercept_
slope = model.coef_[0]

print("Intercept:", intercept)
print("Slope:", slope)

In [ ]:
# Box plots for the 20 most common deparments in the dataset
import plotly.express as px

clean_df = dep_size_df[['Department', 'Rating Sentiment']].dropna()
df_with_departments = clean_df[clean_df['Department'] != 'Not Found']

top_departments = (
    df_with_departments['Department']
    .value_counts()
    .nlargest(20)
    .index)

filtered_df = df_with_departments[df_with_departments['Department'].isin(top_departments)]

filtered_df['Department'] = pd.Categorical(
    filtered_df['Department'],
    categories=top_departments,
    ordered=True)

fig = px.box(
    filtered_df,
    x='Department',
    y='Rating Sentiment',
    title='Sentiment Distribution for Top 20 Departments',
    labels={'Department': 'Department', 'Rating Sentiment': 'Sentiment'})

fig.update_layout(xaxis_tickangle=-45)
fig.show()